# Reading in Data From Matlab

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io as sio
import seaborn as sns
%matplotlib inline

In [2]:
spurs1 = sio.loadmat('spurs1_sss_precip_wind.mat')
spurs2 = sio.loadmat('spurs2_sss_precip_wind.mat')

In [3]:
df = pd.DataFrame(np.hstack(spurs1['spurs1_data'][:,0]))
df

,time,sss,sst,precip,wind_spd,wind_dir,cum_precip3,cum_precip6
0,"[[735126.8541666667], [735126.8958333334], [73...","[[37.76821725535747], [37.7672907861594], [37....","[[27.88650273367723], [27.79417054791615], [27...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[3.171526517031759], [3.5115902949650715], [4...","[[-70.40256567460615], [-78.65223135098692], [...","[[nan], [nan], [0.0], [0.0], [0.0], [0.0], [0....","[[nan], [nan], [nan], [nan], [nan], [0.0], [0...."


In [4]:
df2 = pd.DataFrame(np.hstack(spurs2['spurs1_data'][:,0]))
df2

,time,sss,sst,precip,wind_spd,wind_dir,cum_precip3,cum_precip6
0,"[[736566.1041666667], [736566.1458333334], [73...","[[32.78581320837813], [32.78952749432496], [32...","[[29.972727771803747], [30.088739078063007], [...","[[0.0], [0.0], [10.613046646917608], [1.210006...","[[1.4993049364106], [2.2011608470558626], [4.6...","[[-35.279577496102746], [-44.30044423240883], ...","[[nan], [nan], [10.613046646917608], [11.82305...","[[nan], [nan], [nan], [nan], [nan], [11.823053..."


Making a function to read in all the data...

In [6]:
def extract(col):
    y = []
    for i in range(0, 9133):
        y.append(df[col][0][i][0])
    return y

In [7]:
def extract2(col):
    y = []
    for i in range(0, 10459):
        y.append(df2[col][0][i][0])
    return y

In [8]:
## For Spurs1:
time = extract('time')
sst = extract('sst')
sss = extract('sss')
precip = extract('precip')
wind_spd = extract('wind_spd')
wind_dir = extract('wind_dir')
cum_precip3 = extract('cum_precip3')
cum_precip6 = extract('cum_precip6')

## For Spurs2:
time2 = extract2('time')
sst2 = extract2('sst')
sss2 = extract2('sss')
precip2 = extract2('precip')
wind_spd2 = extract2('wind_spd')
wind_dir2 = extract2('wind_dir')
cum_precip32 = extract2('cum_precip3')
cum_precip62 = extract2('cum_precip6')

In [10]:
# Creating a dataframe for each set
data = pd.DataFrame([time, sss, sst, wind_spd, wind_dir, precip, cum_precip3, cum_precip6])
data2 = pd.DataFrame([time2, sss2, sst2, wind_spd2, wind_dir2, precip2,  cum_precip32, cum_precip62])

# Putting it into the proper layout
data = data.transpose()
data2 = data2.transpose()

# ... and finally naming the columns
data.columns = (['time', 'sss', 'sst', 'wind_spd', 'wind_dir', 'precip', 'cum_precip3', 'cum_precip6'])
data2.columns = (['time', 'sss', 'sst', 'wind_spd', 'wind_dir', 'precip', 'cum_precip3', 'cum_precip6'])

data2.head(1)

,time,sss,sst,wind_spd,wind_dir,precip,cum_precip3,cum_precip6
0,736566.104167,32.785813,29.972728,1.499305,-35.279577,0.0,NaN,NaN


We were somewhat suspicious of the cumulative precipitations, so we decided to compute them ourselves as a rolling sum.

In [14]:
data.drop(['cum_precip3', 'cum_precip6'], axis=1, inplace=True)
data2.drop(['cum_precip3', 'cum_precip6'], axis=1, inplace=True)

data['cum_precip3'] = data['precip'].rolling(3).sum()
data['cum_precip6'] = data['precip'].rolling(6).sum()

data2['cum_precip3'] = data2['precip'].rolling(3).sum()
data2['cum_precip6'] = data2['precip'].rolling(6).sum()

data2.describe()

,time,sss,sst,wind_spd,wind_dir,precip,cum_precip3,cum_precip6
count,10459.000000,10459.000000,10459.000000,10459.000000,10459.000000,10459.000000,1.045700e+04,1.045400e+04
mean,736783.979167,33.425595,27.863688,5.782318,-44.975005,0.375486,1.126673e+00,2.250715e+00
std,125.808232,0.673548,0.619148,2.235571,107.111796,1.927136,4.386714e+00,7.054332e+00
min,736566.104167,30.702347,26.630900,0.007352,-179.936301,0.000000,-7.993606e-15,-1.776357e-14
25%,736675.041667,32.915935,27.251241,4.235575,-138.433271,0.000000,0.000000e+00,-9.769963e-15
50%,736783.979167,33.369679,27.996272,6.136603,-108.354818,0.000000,1.776357e-15,2.708944e-14
75%,736892.916667,34.078227,28.324040,7.516339,56.066751,0.000000,6.622575e-02,5.398891e-01
max,737001.854167,34.398257,30.498667,12.201906,179.926803,45.403696,9.143455e+01,1.105281e+02


In [15]:
data.fillna(0, inplace=True)
data2.fillna(0, inplace=True)

## Inspecting Our Data

We were told that we classify a "rain event" as 2 mm or more over the course of 3 hours. How often do we see one?

In [16]:
print(data[data['cum_precip3'] > 2].size/8)

print(data2[data2['cum_precip3'] > 2].size/8)

214.0
1127.0


... so very rare. Only about 2.3% and 10.7% for each set, respectively.

# Feature Engineering

We will definitely need more than just these variables. Some of the things we decided that we may need include temporal data, and the amount of change in our current variables per hour.

## Adding Time

In [17]:
# For Spurs1
import datetime
from matplotlib import dates as dates
fix_time = pd.to_datetime(data['time']-719529, unit='D')
fix_time.head()

data['new_time'] = fix_time
data['year'] = pd.DatetimeIndex(data['new_time']).year
data['month'] = pd.DatetimeIndex(data['new_time']).month
data['day'] = pd.DatetimeIndex(data['new_time']).day
data['hour'] = pd.DatetimeIndex(data['new_time']).hour
data = data.set_index('new_time')


# For Spurs2
import datetime
from matplotlib import dates as dates
fix_time2 = pd.to_datetime(data2['time']-719529, unit='D')
fix_time2.head()

data2['new_time'] = fix_time2
data2['year'] = pd.DatetimeIndex(data2['new_time']).year
data2['month'] = pd.DatetimeIndex(data2['new_time']).month
data2['day'] = pd.DatetimeIndex(data2['new_time']).day
data2['hour'] = pd.DatetimeIndex(data2['new_time']).hour
data2 = data2.set_index('new_time')

Now we add in our changes between hours. We decided to look at the percentage change from one hour to the next for now, though we may change this later. Of course, for our models we'll also need to scale everything.

## Adding Scaled Columns and Change

In [18]:
from sklearn.preprocessing import StandardScaler, scale
sc = StandardScaler()
# For Spurs1
scaled_stuff = sc.fit_transform(data['sss'].values.reshape(-1,1))
scaled_stuff2 = sc.fit_transform(data['sst'].values.reshape(-1,1))
scaled_stuff3 = sc.fit_transform(data['wind_dir'].values.reshape(-1,1))
scaled_stuff4 = sc.fit_transform(data['wind_spd'].values.reshape(-1,1))
data['scaled_sss'] = scaled_stuff
data['scaled_dir'] = scaled_stuff3
data['scaled_spd'] = scaled_stuff4
data['scaled_sst'] = scaled_stuff2
data['pct_change'] = data['sss'].pct_change()
scaled_stuff5 = sc.fit_transform(data['pct_change'].values.reshape(-1,1))
data['pct_c_scaled'] = scaled_stuff5
data['set'] = 'Spurs1'
data['sst_pct_change'] = scale(data['sst'].pct_change())
data['dir_pct_change'] = scale(data['wind_dir'].pct_change())
data['spd_pct_change'] = scale(data['wind_spd'].pct_change())

# For Spurs2
scaled_stuff = sc.fit_transform(data2['sss'].values.reshape(-1,1))
scaled_stuff2 = sc.fit_transform(data2['sst'].values.reshape(-1,1))
scaled_stuff3 = sc.fit_transform(data2['wind_dir'].values.reshape(-1,1))
scaled_stuff4 = sc.fit_transform(data2['wind_spd'].values.reshape(-1,1))
data2['scaled_sss'] = scaled_stuff
data2['scaled_dir'] = scaled_stuff3
data2['scaled_spd'] = scaled_stuff4
data2['scaled_sst'] = scaled_stuff2
data2['pct_change'] = data2['sss'].pct_change()
scaled_stuff5 = sc.fit_transform(data2['pct_change'].values.reshape(-1,1))
data2['pct_c_scaled'] = scaled_stuff5
data2['set'] = 'Spurs2'
data2['sst_pct_change'] = scale(data2['sst'].pct_change())
data2['dir_pct_change'] = scale(data2['wind_dir'].pct_change())
data2['spd_pct_change'] = scale(data2['wind_spd'].pct_change())

data.fillna(0, inplace=True)
data2.fillna(0, inplace=True)

# PCA and RFE

The last thing we need to do before we begin running models is to find which variables matter the most. We do not simply want to throw everything into each model, as this will create far too much noise and worsen our results. Therefore, we decided to find out how few variables we need to explain the majority of the variance in our data.

## For Spurs1

In [20]:
#data.info()

In [24]:
X = data.loc[:, ['day','month','hour','scaled_dir','scaled_spd','scaled_sst','scaled_sss','pct_c_scaled']]
y = np.array([1 if i>2.0 else 0 for i in data.iloc[:,5]])
y

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X)
csum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(csum>=.98)+1
print(d)
csum

5


array([0.54312222, 0.88269379, 0.97019278, 0.97789321, 0.9851118 ,
       0.99183863, 0.99718151, 1.        ])

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
logistic = SGDClassifier(loss='log', penalty='l2', early_stopping=True,
                         max_iter=10000, tol=1e-5, random_state=0)
pca = PCA(.98)
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [1,2,3,4,5,6,7,8],
    'logistic__alpha': np.logspace(-4, 4, 10),
}
search = GridSearchCV(pipe, param_grid, iid=False, cv=5,
                      return_train_score=False)
search.fit(X, y.ravel())
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)
pca.fit(X)
print(pd.DataFrame(pca.components_,columns=X.columns))
pca.explained_variance_ratio_

Best parameter (CV score=0.991):
{'logistic__alpha': 0.005994842503189409, 'pca__n_components': 8}
        day     month      hour  scaled_dir  scaled_spd  scaled_sst  \
0  0.999524  0.014559 -0.003999    0.009547   -0.019250    0.008071   
1  0.004048  0.000374  0.999938   -0.008409    0.000417    0.005874   
2  0.012455 -0.968194  0.001806    0.020094   -0.066621   -0.202176   
3  0.000474  0.054347 -0.003577   -0.545587    0.136864   -0.043040   
4 -0.016360  0.049679  0.006095    0.648954   -0.462589    0.003008   

   scaled_sss  pct_c_scaled  
0   -0.014003     -0.000126  
1    0.000839      0.000645  
2   -0.129319     -0.001418  
3   -0.487601     -0.664100  
4   -0.032398     -0.600865  


array([0.54312222, 0.33957157, 0.08749899, 0.00770043, 0.00721859])

So this suggests that wind direction, salinity change, and wind speed are the most important factors by a decent margin.

In [40]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 5)
rfe = rfe.fit(X, y.ravel())
print(rfe.support_)
print(rfe.ranking_)
X.head(1)

[False False False  True  True  True  True  True]
[4 2 3 1 1 1 1 1]


,day,month,hour,scaled_dir,scaled_spd,scaled_sst,scaled_sss,pct_c_scaled
new_time,,,,,,,,
2012-09-14 20:30:00.000028800,14,9,20,-0.223661,-1.120622,2.145583,2.845093,0.0


... and this confirms it. Temporal data is important but this suggests that the variables (and the interactions between them) have a more direct effect for now.

## Spurs2

In [44]:
X2 = data2.loc[:, ['day','month','hour','scaled_dir','scaled_spd','scaled_sst','scaled_sss','pct_c_scaled']]
y2= np.array([1 if i>2.0 else 0 for i in data2.iloc[:,5]])
y2

array([0, 0, 1, ..., 0, 0, 0])

In [45]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X2)
csum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(csum>=.98)+1
print(d)
csum

4


array([0.55279943, 0.88725222, 0.97390722, 0.98218272, 0.98914158,
       0.99420537, 0.99797221, 1.        ])

So this suggests that we only need 4 variables to explain over 98% of our data

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
logistic = SGDClassifier(loss='log', penalty='l2', early_stopping=True,
                         max_iter=10000, tol=1e-5, random_state=0)
pca = PCA(.98)
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [1,2,3,4,5,6,7,8],
    'logistic__alpha': np.logspace(-4, 4, 10),
}
search = GridSearchCV(pipe, param_grid, iid=False, cv=5,
                      return_train_score=False)
search.fit(X2, y2.ravel())
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)
pca.fit(X2)
print(pd.DataFrame(pca.components_,columns=X2.columns))
pca.explained_variance_ratio_

Best parameter (CV score=0.954):
{'logistic__alpha': 0.046415888336127774, 'pca__n_components': 8}
        day     month      hour  scaled_dir  scaled_spd  scaled_sst  \
0  0.999971  0.004640  0.001888    0.003536   -0.004453   -0.000041   
1 -0.001892 -0.000160  0.999992    0.002561    0.000995    0.002281   
2  0.005627 -0.943286  0.000311   -0.146850    0.126067   -0.140096   
3 -0.003010 -0.276909 -0.001781    0.409052   -0.597758    0.560356   

   scaled_sss  pct_c_scaled  
0   -0.000762     -0.000273  
1    0.000515     -0.000515  
2    0.230422     -0.000125  
3   -0.205198     -0.206315  


array([0.55279943, 0.33445279, 0.086655  , 0.0082755 ])

So this appears to tell us that wind direction, wind speed, temperature, and month are the most important? Let's see what RFE tells us.

In [47]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 4)
rfe = rfe.fit(X2, y2.ravel())
print(rfe.support_)
print(rfe.ranking_)
X2.head(1)

[False False False  True  True False  True  True]
[5 3 4 1 1 2 1 1]


,day,month,hour,scaled_dir,scaled_spd,scaled_sst,scaled_sss,pct_c_scaled
new_time,,,,,,,,
2016-08-24 02:30:00.000028800,24,8,2,0.090521,-1.915939,3.406521,-0.949913,0.0


RFE, however, suggests to us that the wind direction and speed, as well as the salinity and its change are the most important.

__One thing to remember is that this data is clearly not linear; so, while PCA is useful, we should be wary of it. This is why we also look at RFE, which is a logistic method, and make judgements off of both.__

In [49]:
#data.to_csv('data.csv')
#data2.to_csv('data2.csv')